In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import shutil
import random
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
from plotly.colors import n_colors
from plotly.subplots import make_subplots
import plotly
import pytz
from datetime import datetime, timezone
tz = pytz.timezone('Europe/Berlin')
mynow = datetime.now(tz).strftime('%H:%M:%S')
import yfinance
import warnings
warnings.filterwarnings('ignore')
import os


In [ ]:
btc_historical = pd.read_csv('./BTC_daily_historical.csv')
btc_historical = btc_historical[btc_historical['volume'] > 0]
btc_historical['mav'] = btc_historical['close'].rolling(7).mean().values
btc_historical.dropna(inplace=True)
btc_historical.reset_index(inplace=True, drop=True)

btc_historical.head(5)

In [ ]:
date_format = "%Y-%m-%dT%H:%M:%SZ"
def ohlc_creator(crypto, btch1, ohlclw = 2, lw = 2, ts = 0.5, vlb = 0.4, moving_avg=True, split_ratio=0.8):
    days = 7
    btch1['date'] = pd.to_datetime(btch1['date']).apply(lambda x: str(x.date()))
    btch1.index = btch1['date']
    
                   
    if moving_avg == True:
        foldername= '{}{}_rgb_ohlclw{}_ts{}_lw{}_vlb{}'.format(crypto, days, str(ohlclw), str(ts).replace('.', ''), str(lw), str(vlb).replace('.', ''))
    else:
        foldername= '{}{}_rgb_ohlclw{}_ts{}_lw{}_vlb00'.format(crypto, days, str(ohlclw), str(ts).replace('.', ''), str(lw))

        
    # display(btch1.head(), foldername, '  ', datetime.now(tz).strftime('%H:%M:%S'))
    
    # try:
    #     os.mkdir('./etfs/etf7/{}'.format(foldername))
    # except:
    #     pass
    
    
    for j, i in enumerate(range(0, len(btch1)-days, 1)):
        if j < 3:
            begin = str(btch1.index[i]).split(' ')[0]
            end = str(btch1.index[i+days-1]).split(' ')[0]
            img_name = './etfs/etf7/{}/{}-{}_{}.jpeg'.format(foldername, crypto, begin, end)       
            
            fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.75, 0.25], vertical_spacing = 0)
            fig.add_trace(go.Ohlc(x=btch1.iloc[i:i+days].index, open=btch1.iloc[i:i+days].open, high=btch1.iloc[i:i+days].high, low=btch1.iloc[i:i+days].low,
                            close=btch1.iloc[i:i+days].close, increasing_line_color= 'red', decreasing_line_color= 'red'), row=1, col=1)
            if moving_avg == True:
                fig.add_trace(go.Scatter(x=btch1.iloc[i:i+days].index, y=btch1.iloc[i:i+days].mav, opacity=1, line=dict(color='green', width=lw)), row=1, col=1)

            fig.add_trace(go.Bar(x=btch1.iloc[i:i+days].index, y=btch1.iloc[i:i+days].volume, marker_color='blue'), row=2, col=1)

            fig.update(layout_xaxis_rangeslider_visible=False)
            fig.update_layout(paper_bgcolor='black', plot_bgcolor='black', showlegend=False, autosize=False, width=800, height=400, bargap = vlb, margin=dict(l=0, r=0, t=0, b=0))#, xaxis=dict(margin_pad=2)
            fig.update_yaxes(showgrid=False, showticklabels=False) #, showticklabels=False
            fig.update_xaxes(showgrid=False, ticklen=1, showticklabels=False)
            fig.update_traces(line_width=ohlclw, tickwidth=ts, selector=dict(type='ohlc'))
            # fig.write_image("{}".format(img_name), engine="kaleido", scale=1)
            # print(img_name)
            fig.show()

for i in [4, 6, 10, 12, 14][3:4]:
    for j in [2, 3][:1]:
        ohlc_creator('BTC', btc_historical.copy(), ohlclw = i, ts = 0.4, lw = j,  vlb = 0.5, moving_avg=True, split_ratio=0.8)